# Episodio 11: definire nuovi tipi, parte 2
Gli esempi qui sotto possono essere eseguiti copiandoli nell'interprete di Haskell, il `ghci`. Se vuoi eseguire direttamente questo notebook, le istruzioni su come farlo sono disponibili a [questo](https://github.com/gibiansky/IHaskell) link.

## Precisazioni sui record types
### 1. I nomi dei "campi" di un record sono funzioni top-level
...E per funzioni top-level s'intende funzioni con visibilità globale, il che significa che una volta definito un record type, __non__ possiamo definire altre funzioni che hanno lo stesso nome di uno dei suoi campi, né altri record con campi con lo stesso nome. 
Provando a farlo, otterremo un errore a tempo di compilazione.
Per capire meglio, utilizziamo un tipo definito la scorsa volta, `Persona` (che ci richiede anche di ridefinire un altro tipo, `Sesso`):

In [1]:
data Persona = Persona {
    nome :: String,
    eta :: Int,
    sesso :: Sesso } 
    deriving Show 
    
data Sesso = Maschio | Femmina
    deriving Show
    
nome :: Persona -> String
nome = undefined

data AnimaleDomestico = AnimaleDomestico {
    nome :: String,
    specie :: String }

: 

### 2. Lasciare un campo non inizializzato è una cattiva idea
Se __non__ inizializziamo un campo, come nel seguente esempio, otterremo invece un semplice warning:

In [2]:
d = Persona {
    nome = "Davide",
    eta = 20 }

: 

... o almeno, così si comporta il mio `ghci`. A quanto pare, i notebook sono ancora più severi! In `ghci`, ad ogni modo, inizializzare `d` come sopra sarebbe tutto sommato valido. Il problema è che chiamare `sesso d` risulterebbe in un temutissimo errore a tempo di esecuzione (output della mia console):

```
ghci> sesso d
*** Exception: Ep11.hs:(21,5)-(23,14): Missing field in record construction 
```

Attenzione quindi a non aspettarsi un semplice `NULL` in risposta e a non fare mai niente del genere.

## Tipi parametrici
L'errore di cui sopra può forse far sorgere il dubbio che il riferimento a `NULL` presente nella stragrande maggioranza dei linguaggi di programmazione non sia un'idea tanto cattiva. Ovviamente però _lo è_, e in Haskell possiamo trovare una soluzione migliore per indicare l'assenza di un valore definendo un _tipo parametrico_.

Un tipo parametrico è un tipo il cui costruttore (o i cui costruttori) contiene (o contengono) un parametro. Attenzione: per _costruttore di un tipo_ intendiamo quello che si trova __a sinistra__ dell' `=`.  

Per esempio:

In [3]:
data Forse a = Proprio a | Niente
    deriving Show -- ci servirà dopo per poter "vedere" questo tipo istanziato

In questo esempio, `Niente` indica l'assenza di un valore, costruendo un valore "nullo". Una formulazione alternativa potrebbe essere quella in cui definiamo un tipo forse "pieno" e forse "vuoto":

In [4]:
data Forse' a = Pieno a | Vuoto
    deriving Show

Usando `Forse`, possiamo ridefinire `Persona` come segue: 

In [5]:
data Persona' = Persona' {
    nome' :: String,
    eta' :: Int,
    sesso' :: Forse Sesso }
    deriving Show 
    
data Sesso = Maschio | Femmina
    deriving Show

E finalmente creare una `Persona` dal `Sesso` non specificato:

In [6]:
d' = Persona' {
    nome' = "Davide",
    eta' = 20,
    sesso' = Niente }

### Tipi parametrici "illustri"
Alcuni tipi parametrici ampiamente utilizzati in Haskell sono i seguenti:
- [`Maybe`](https://hackage.haskell.org/package/base-4.15.0.0/docs/Data-Maybe.html), di cui `Forse` è la "traduzione italiana"
- [`Either`](https://hackage.haskell.org/package/base-4.15.0.0/docs/Data-Either.html), per certi versi simile a `Maybe`, spesso usato per la gestione degli errori
- [`Map`](https://hackage.haskell.org/package/containers-0.6.5.1/docs/Data-Map.html), molto simile ai [dizionari di Python](https://docs.python.org/3/tutorial/datastructures.html#dictionaries)
- ...e le liste, che semplicemente hanno un costruttore "curioso" (le parentesi quadre)

## Altre parole chiave per la definizione di nuovi tipi

### `type`
`type` serve a definire _type synonym_, ossia (letteralmente!) __sinonimi di tipi preesistenti__ (anche definiti da noi nello stesso modulo). E' spesso utile per distinguere stringhe (o altro) con usi diversi. Per esempio, pensando all'esempio precedente potremmo voler definire i seguenti sinonimi:

In [7]:
type Nome = String
type Cognome = String
type Eta = Int

`Persona` potrebbe quindi essere ri-ri-ri-ri definito come:

In [8]:
data Persona''' = Persona''' {
    nome''' :: Nome,
    cognome :: Cognome,
    eta''' :: Eta,
    sesso''' :: Maybe Sesso } -- usando la versione "standard" di Forse
    deriving Show
    
data Sesso = Maschio | Femmina
    deriving Show

### `newtype`
Con `newtype` possiamo invece definire __nuovi tipi con un solo costruttore e un solo parametro__, come

In [9]:
newtype Gruppo a = Gruppo [a]

A prima vista, questo può sembrare inutile, ma ha senso se vogliamo ad es. definire comportamenti particolari per, in questo caso, un certo tipo di liste (ad es. per istanziarne uno `Show` personalizzato). Vedremo presto come farlo.

Chiaramente, nulla ci impedisce di utilizzare il caro vecchio `data`. Il comportamento sarà lo stesso dal p.d.v. della logica ma il vostro linter (sempre che ne abbiate installato uno) vi suggerirà di utilizzare type per questioni di ottimizzazione:

In [10]:
data Gruppo a = Gruppo [a]

Line 1: Use newtype instead of data
Found:
data Gruppo a = Gruppo [a]
Why not:
newtype Gruppo a = Gruppo [a]

Dalla Haskell Wiki (articolo completo disponibile [qui](wiki.haskell.org/Newtype)) in traduzione, per chi fosse interessato e avesse già chiaro il concetto di type checker:
   
   > La restrizione a un solo costruttore con un solo campo assicura che il 
   nuovo tipo e il tipo del suo campo siano in corrispondenza diretta, o in
   termini matematici che essi [il tipo e il suo campo] siano isomorfi.
   Ciò significa che, dopo il type checking a tempo di compilazione, a tempo
   di esecuzione i due tipi possono essere trattati sostanzialmente nello
   stesso modo [...]. Quindi, se l'obiettivo è dichiarare diverse istanze di
   una type class per un certo tipo, lo si può "avvolgere" [wrap] in un
   newtype in modo che sia considerato come un tipo distinto dal 
   type-checker, ma come identico al suo campo a runtime. 